In [4]:
import numpy as np

from time import time
import os
from os import listdir
from os.path import isfile, join
from sys import getsizeof
from collections import Mapping, Container

import glob
import string
import math
from nltk.stem.porter import *

In [5]:
def directory_listing(dir_path):
    '''
    Input:  string (path to directory)
    Output: list of strings (full paths to files in the directory)
    '''
    
    return glob.glob(dir_path + '*.txt')

In [29]:
path = os.getcwd() +'/HillaryEmails/'
all_files = directory_listing(path)
print(all_files)
#all_files

['/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/3644.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/5235.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/1053.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/7422.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/7344.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/1735.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/5553.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/4895.txt', '/Users/arkapravasaha/Documents/CI6226/Assignment/Information-Retrieval-and-Analysis/HillaryEmails/3122.txt', '/Users/a

In [6]:
def read_file(file_path):
    '''
    Input:  string (full path to file)
    Output: string/text (full contents of a file)
    '''
    
    lines = [line.rstrip('\n') for line in open(file_path)]
    return ' '.join(lines)

In [31]:
file_text = read_file(all_files[0])
file_text

'UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05766459 Date: 07/31/2015 RELEASE IN FULL CONFIDENTIAL October 23, 2009 For: Hillary From: Sid Re: Tony Blair, EU presidency, Tory Party, & Berlin trip One of your agenda items behind the scenes on your Berlin trip can be to discuss the future of the EU, the European presidency and the prospects of Tony Blair. If Blair does not become EU president the position will likely be filled by .a third rank nonentity in the Brussels bureaucratic mode incapable of realizing the possibilities in the creation of the office, continuing the feebleness of Europe as a political idea and reality. Of course, it is in the US interest to have a strong Europe—and the naming of the first European president might be the most important opportunity for the US to strengthen Europe, to give it actual sinew, for a long time and a long time to come. The Conservative Party is split over Europe and Blair. Hague represents the Tory right, which is 

In [7]:
def tokenization(text, file_path):
    '''
    Input:  text(file contents), string (document id = path to file)
    Output: list of pairs < string(token) , string (document id) >
    '''
    doc_id = int(os.path.basename(file_path).replace('.txt','')) #retrieve document id from file path
    tokens = text.split()
    return [(token, doc_id) for token in tokens]
    

In [34]:
token_pairs = tokenization(file_text, all_files[0])
print("length of pairs: ", len(token_pairs))
print(token_pairs)
#token_pairs

length of pairs:  605
[('UNCLASSIFIED', 3644), ('U.S.', 3644), ('Department', 3644), ('of', 3644), ('State', 3644), ('Case', 3644), ('No.', 3644), ('F-2014-20439', 3644), ('Doc', 3644), ('No.', 3644), ('C05766459', 3644), ('Date:', 3644), ('07/31/2015', 3644), ('RELEASE', 3644), ('IN', 3644), ('FULL', 3644), ('CONFIDENTIAL', 3644), ('October', 3644), ('23,', 3644), ('2009', 3644), ('For:', 3644), ('Hillary', 3644), ('From:', 3644), ('Sid', 3644), ('Re:', 3644), ('Tony', 3644), ('Blair,', 3644), ('EU', 3644), ('presidency,', 3644), ('Tory', 3644), ('Party,', 3644), ('&', 3644), ('Berlin', 3644), ('trip', 3644), ('One', 3644), ('of', 3644), ('your', 3644), ('agenda', 3644), ('items', 3644), ('behind', 3644), ('the', 3644), ('scenes', 3644), ('on', 3644), ('your', 3644), ('Berlin', 3644), ('trip', 3644), ('can', 3644), ('be', 3644), ('to', 3644), ('discuss', 3644), ('the', 3644), ('future', 3644), ('of', 3644), ('the', 3644), ('EU,', 3644), ('the', 3644), ('European', 3644), ('presidency'

In [8]:
def linguistic_modules(token_pairs):
    '''
    Input:  list of pairs < token , document id >
    Output: list of pairs < modified token , document id >
    
    modified token: removing all punctuation symbols (!@#$%^&*()-_=+’`~”:;/.,?[]{}<>),lowercasingand stemming.
    '''
    
    stemmer = PorterStemmer()  
    return [(stemmer.stem(token.translate(str.maketrans('','',string.punctuation)).lower()), doc_id)  
            for token, doc_id in token_pairs  
                if token.translate(str.maketrans('','',string.punctuation)) is not ''] #if statement to check empty token
    

In [36]:
modified_token_pairs = linguistic_modules(token_pairs)
print('length after modification: ', len(modified_token_pairs))
print(modified_token_pairs)
#modified_token_pairs

length after modification:  604
[('unclassifi', 3644), ('us', 3644), ('depart', 3644), ('of', 3644), ('state', 3644), ('case', 3644), ('no', 3644), ('f201420439', 3644), ('doc', 3644), ('no', 3644), ('c05766459', 3644), ('date', 3644), ('07312015', 3644), ('releas', 3644), ('in', 3644), ('full', 3644), ('confidenti', 3644), ('octob', 3644), ('23', 3644), ('2009', 3644), ('for', 3644), ('hillari', 3644), ('from', 3644), ('sid', 3644), ('re', 3644), ('toni', 3644), ('blair', 3644), ('eu', 3644), ('presid', 3644), ('tori', 3644), ('parti', 3644), ('berlin', 3644), ('trip', 3644), ('one', 3644), ('of', 3644), ('your', 3644), ('agenda', 3644), ('item', 3644), ('behind', 3644), ('the', 3644), ('scene', 3644), ('on', 3644), ('your', 3644), ('berlin', 3644), ('trip', 3644), ('can', 3644), ('be', 3644), ('to', 3644), ('discuss', 3644), ('the', 3644), ('futur', 3644), ('of', 3644), ('the', 3644), ('eu', 3644), ('the', 3644), ('european', 3644), ('presid', 3644), ('and', 3644), ('the', 3644), ('p

In [67]:
l = [('unclassifi', '133'),('us', '133'),('depart', '133'), ('!','12')]
linguistic_modules(l)

[('unclassifi', '133'), ('us', '133'), ('depart', '133')]

In [9]:
def sort_tokens(token_pairs):
    '''
    Input:  list of pairs < token , document id >
    Output: sorted list of pairs < token , document id >
    
    perform sorting of the token list: first by tokens (alphabetical order), and then by document ids 
    '''
    
    return sorted(token_pairs, key=lambda element: (element[0], element[1]))

In [38]:
sorted_modified_token_pairs = sort_tokens(modified_token_pairs)
sorted_modified_token_pairs

[('07312015', 3644),
 ('07312015', 3644),
 ('07312015', 3644),
 ('07312015', 3644),
 ('1', 3644),
 ('2', 3644),
 ('2009', 3644),
 ('23', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('a', 3644),
 ('abl', 3644),
 ('about', 3644),
 ('about', 3644),
 ('act', 3644),
 ('actual', 3644),
 ('affili', 3644),
 ('against', 3644),
 ('agenda', 3644),
 ('align', 3644),
 ('align', 3644),
 ('all', 3644),
 ('also', 3644),
 ('american', 3644),
 ('an', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('and', 3644),
 ('antiamerican', 3644),
 ('antieuropean', 3644),
 ('antieuropean', 3644),
 ('anyth', 3644),
 ('appreci', 3644),
 ('are', 3644),
 ('are', 3644),
 ('are', 3644),
 ('argument', 3644),
 ('aristocrat', 3644),
 ('as', 3644),
 ('as', 3644),
 ('as', 3644),
 ('as', 36

In [10]:
def transformation_into_postings(sorted_token_pairs):
    '''
    Input: sorted list of pairs < token , document id >
    Output: inverted index
    
    Used dictionary data structure (Hash table)
    '''
    
    dictionary_ = {}
    for a, b in sorted_token_pairs:
        dictionary_.setdefault(a, []).append(b)
#     dictionary_ = {key:list(sorted(set(value))) for (key, value) in dictionary_.items()}
    for key in dictionary_:
        value = dictionary_[key]
        posting = list(sorted(set(value)))
        dictionary_[key] = (len(posting),posting)
    return dictionary_

In [40]:
posting_list = transformation_into_postings(sorted_modified_token_pairs)
posting_list

{'07312015': (1, [3644]),
 '1': (1, [3644]),
 '2': (1, [3644]),
 '2009': (1, [3644]),
 '23': (1, [3644]),
 'a': (1, [3644]),
 'abl': (1, [3644]),
 'about': (1, [3644]),
 'act': (1, [3644]),
 'actual': (1, [3644]),
 'affili': (1, [3644]),
 'against': (1, [3644]),
 'agenda': (1, [3644]),
 'align': (1, [3644]),
 'all': (1, [3644]),
 'also': (1, [3644]),
 'american': (1, [3644]),
 'an': (1, [3644]),
 'and': (1, [3644]),
 'antiamerican': (1, [3644]),
 'antieuropean': (1, [3644]),
 'anyth': (1, [3644]),
 'appreci': (1, [3644]),
 'are': (1, [3644]),
 'argument': (1, [3644]),
 'aristocrat': (1, [3644]),
 'as': (1, [3644]),
 'at': (1, [3644]),
 'away': (1, [3644]),
 'balanc': (1, [3644]),
 'be': (1, [3644]),
 'becom': (1, [3644]),
 'behind': (1, [3644]),
 'berlin': (1, [3644]),
 'best': (1, [3644]),
 'bit': (1, [3644]),
 'blair': (1, [3644]),
 'bori': (1, [3644]),
 'britain': (1, [3644]),
 'brussel': (1, [3644]),
 'bureaucrat': (1, [3644]),
 'bush': (1, [3644]),
 'but': (1, [3644]),
 'by': (1, 

In [72]:
a = [483, 483, 1526, 1526, 1840, 1840, 1840, 1840]
list(set(a))

[1840, 483, 1526]

In [73]:
d = {'a': [1,1,2], 'c': [3], 'b': [2,3,2]}
{key:list(set(value)) for (key, value) in d.items()}


{'a': [1, 2], 'c': [3], 'b': [2, 3]}

In [74]:
dict1 = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}
# Double each value in the dictionary
double_dict1 = {k:v*2 for (k,v) in dict1.items()}
print(double_dict1)

{'a': 2, 'b': 4, 'c': 6, 'd': 8, 'e': 10}


In [11]:
def postings_list_merge(postings_lists):
    '''
    Input: list of postings lists 
    Output: merged postings list
    
    Intersect the postings lists in increasing order of length
    '''
    
    sorted_postings_lists = sorted(postings_lists, key = lambda l : l[0], reverse = True)
    first_len, first_list = sorted_postings_lists.pop()
    while sorted_postings_lists:
        second_len, second_list = sorted_postings_lists.pop()
        merged_list = []
        p1 = 0
        p2 = 0
        length = 0
        while p1 < first_len and p2 < second_len:
            if first_list[p1] == second_list[p2]:
                merged_list.append(first_list[p1])
                p1 += 1
                p2 += 1
                length += 1
            elif first_list[p1] < second_list[p2]:
                p1 += 1
            else:
                p2 += 1
        first_list = merged_list
        first_len = length
    return merged_list

In [28]:
def tfidf_weight(token,docid,term_freq,posting_list):
    if token not in term_freq[docid]:
        return 0
    tf = 1+math.log10(term_freq[docid][token])
    idf = math.log10((len(term_freq)-1)/posting_list[token][0])
    return (tf*idf)

In [12]:
# def tfidf_score(query_tokens,docid,tfidf,posting_list):
def tfidf_score(query_tokens,docid,tfidf):
    '''
    Input: tokens in query, document id, term frequencies
    Output: tf-idf score
    
    Compute tf-idf score of a document for a query
    '''
    
    score = 0
    for token in query_tokens:
#         score += tfidf_weight(token,docid,term_freq,posting_list)
        score += tfidf[docid][token]
    return score

In [13]:
def cosine_score(doc,tfidf,length):
    score = 0
    for term in tfidf[0]:
        if term in tfidf[doc]:
            score += tfidf[0][term]*tfidf[doc][term]
    score /= (length[0]*length[doc])
    return score

In [14]:
# MAIN
start = time()

path = os.getcwd() +'/HillaryEmails/'
# print(path)
all_files = directory_listing(path)
# print(all_files)
#all_files = all_files[:10]

all_token_pairs = []

for file in all_files:
    file_text = read_file(file)
    token_pairs = tokenization(file_text, file)
    modified_token_pairs = linguistic_modules(token_pairs)
    all_token_pairs = all_token_pairs + modified_token_pairs

end = time()
print(end - start)

freq = [{} for i in range(len(all_files)+1)]
for token,docid in all_token_pairs:
    if token not in freq[docid]:
        freq[docid][token] = 0
    freq[docid][token] += 1

440.98028802871704


In [15]:
sorted_tokens = sort_tokens(all_token_pairs)

In [16]:
start = time()

posting_list = transformation_into_postings(sorted_tokens)
# posting_list

end = time()
print(end - start)

2.285060167312622


In [17]:
def deep_getsizeof(o, ids):
    d = deep_getsizeof
    if id(o) in ids:
        return 0
    r = getsizeof(o)
    ids.add(id(o))
    r = getsizeof(o)
    ids.add(id(o))
    if isinstance(o, str):
        return r
    if isinstance(o, Mapping):
        return r + sum(d(k, ids) + d(v, ids) for k, v in o.items())
    if isinstance(o, Container):
        return r + sum(d(x, ids) for x in o)
    return r

In [19]:
mem = deep_getsizeof(posting_list,set())
for tok in posting_list:
    mem -= deep_getsizeof((),set())
    mem -= deep_getsizeof([],set())
mem
# deep_getsizeof({},set())

20571393

In [20]:
for term in posting_list:
    if posting_list[term][0]>=5000:
        print(term,posting_list[term][0])

and 5388
case 7944
date 7944
depart 7944
doc 7944
f201420439 7648
from 7822
h 5177
in 7935
no 7944
of 7944
releas 7913
sent 7670
state 7944
subject 7491
the 5881
to 7857
unclassifi 7944
us 7944


In [21]:
for doc in range(1,len(freq)):
    for term in freq[doc]:
        tf = 1+math.log10(freq[doc][term])
        idf = math.log10((len(freq)-1)/posting_list[term][0])
        freq[doc][term] = tf*idf

In [22]:
length = [0]
for i in range(1,len(freq)):
    length.append(np.linalg.norm(np.array(list(freq[i].values()))))

In [14]:
postings_list_merge([posting_list['mail'],posting_list['phone'],posting_list['clinton']])

[1325, 1397]

In [23]:
# User Queries with TF-IDF
stemmer = PorterStemmer()
query = input('Enter your query: ')
while query is not '':
    start = time()
    tokens = query.split()  
    stemmed_tokens = [stemmer.stem(token.translate(str.maketrans('','',string.punctuation)).lower()) 
            for token in tokens
                if token.translate(str.maketrans('','',string.punctuation)) is not '']
    docs = postings_list_merge([posting_list[tok] for tok in stemmed_tokens])
    end = time()
    t1 = time()
    result = sorted(docs,key = lambda d: tfidf_score(set(stemmed_tokens),d,freq))
    t2 = time()
    print('Time taken to retrieve: '+str(end-start))
    print('Time taken to rank: '+str(t2-t1))
    print('Ranked list of files with query: '+str([str(res)+'.txt' for res in result]))
    print()
    query = input('Enter your query: ')

Enter your query: Clinton's email and phone
Time taken to retrieve: 0.0026900768280029297
Time taken to rank: 0.00010395050048828125
Ranked list of files with query: ['2118.txt', '2353.txt', '2355.txt', '3910.txt', '3911.txt', '7907.txt', '2420.txt', '2043.txt', '2536.txt', '132.txt', '126.txt', '133.txt', '4335.txt', '137.txt', '5264.txt', '6030.txt', '6674.txt', '6675.txt', '3280.txt', '7541.txt', '1416.txt', '6823.txt', '6024.txt', '5549.txt', '32.txt', '5288.txt', '5490.txt', '4299.txt', '1397.txt', '5300.txt', '5998.txt', '5991.txt', '665.txt']

Enter your query: Hillary Clinton emails about politics and presidential election
Time taken to retrieve: 0.005220890045166016
Time taken to rank: 7.414817810058594e-05
Ranked list of files with query: ['2119.txt', '3903.txt', '5098.txt', '3263.txt', '126.txt', '132.txt', '133.txt', '137.txt', '4299.txt', '6258.txt', '5789.txt', '6037.txt', '5998.txt']

Enter your query: Nazi Hitler
Time taken to retrieve: 0.00032830238342285156
Time taken

Enter your query: 


In [24]:
# User Queries with Cosine Similarity
stemmer = PorterStemmer()
query = input('Enter your query: ')
while query is not '':
    start = time()
    tokens = query.split()  
    stemmed_tokens = [stemmer.stem(token.translate(str.maketrans('','',string.punctuation)).lower()) 
            for token in tokens
                if token.translate(str.maketrans('','',string.punctuation)) is not '']
    docs = postings_list_merge([posting_list[tok] for tok in stemmed_tokens])
    freq[0] = {}
    for tok in stemmed_tokens:
        if tok not in freq:
            freq[0][tok] = 0
        freq[0][tok] += 1
    for tok in stemmed_tokens:
        tf = 1+math.log10(freq[0][tok])
        idf = math.log10((len(freq)-1)/posting_list[tok][0])
        freq[0][tok] = tf*idf
    length[0] = np.linalg.norm(np.array(list(freq[0].values())))
    end = time()
    t1 = time()
    result = sorted(docs,key = lambda d: cosine_score(d,freq,length))
    t2 = time()
    print('Time taken to retrieve: '+str(end-start))
    print('Time taken to rank: '+str(t2-t1))
    print('Ranked list of files with query: '+str([str(res)+'.txt' for res in result]))
    print()
    query = input('Enter your query: ')

Enter your query: Clinton's email and phone
Time taken to retrieve: 0.00415802001953125
Time taken to rank: 0.00028705596923828125
Ranked list of files with query: ['5490.txt', '5288.txt', '5998.txt', '1397.txt', '5264.txt', '7541.txt', '4299.txt', '137.txt', '133.txt', '132.txt', '7907.txt', '126.txt', '32.txt', '6675.txt', '6030.txt', '6823.txt', '6674.txt', '6024.txt', '5300.txt', '3280.txt', '665.txt', '4335.txt', '2420.txt', '3911.txt', '2355.txt', '5991.txt', '3910.txt', '2353.txt', '1416.txt', '2536.txt', '5549.txt', '2118.txt', '2043.txt']

Enter your query: Hillary Clinton emails about politics and presidential election
Time taken to retrieve: 0.00713801383972168
Time taken to rank: 8.916854858398438e-05
Ranked list of files with query: ['5789.txt', '6258.txt', '6037.txt', '5998.txt', '3263.txt', '5098.txt', '2119.txt', '4299.txt', '3903.txt', '137.txt', '133.txt', '132.txt', '126.txt']

Enter your query: Nazi Hitler
Time taken to retrieve: 0.0008749961853027344
Time taken to 

Enter your query: 
